Accessing futures and options files kept in same directory


In [9]:
import csv
from datetime import datetime
from itertools import islice
from tqdm import tqdm
import pandas as pd
# Open the CSV file and loop through it line by line
num_data=10000 #muber of datapoints to backtest put 0 for all datapoints and n for last n datapoints
futures_data=[]
with open('BANKNIFTY_2017_FUTURES.csv', 'r') as file:
    futures_reader = csv.reader(file)
    for row in futures_reader:
        if row[-2]=='I':
            futures_data.append(row)
    for line in islice(futures_data,5):
        print(line)  # Process each line as needed
#len(futures_data)
futures_data=futures_data[-1*num_data:]

['BANKNIFTY-I.NFO', '2017-01-02', '09:15:59', '18210.05', '18261.2', '18210.0', '18213.85', '47680.0', '2035880.0', 'I', '2017-01-25']
['BANKNIFTY-I.NFO', '2017-01-02', '09:16:59', '18213.35', '18222.15', '18213.35', '18221.9', '24120.0', '2060560.0', 'I', '2017-01-25']
['BANKNIFTY-I.NFO', '2017-01-02', '09:17:59', '18221.85', '18221.85', '18156.15', '18157.3', '34440.0', '2060560.0', 'I', '2017-01-25']
['BANKNIFTY-I.NFO', '2017-01-02', '09:18:59', '18160.0', '18171.8', '18143.0', '18171.8', '34080.0', '2060560.0', 'I', '2017-01-25']
['BANKNIFTY-I.NFO', '2017-01-02', '09:19:59', '18171.9', '18177.4', '18153.5', '18159.0', '16600.0', '2086360.0', 'I', '2017-01-25']


In [10]:
# Open the CSV file and loop through it line by line
options_data=[]
with open('BANKNIFTY_2017_OPTIONS.csv', 'r') as file:
    options_reader = csv.reader(file)
    for row in options_reader:
        if row[-1]=='I':
            options_data.append(row)
    for line in islice(options_data,5):
        print(line)  # Process each line as needed
options_dataframe = pd.DataFrame(options_data, columns=['Ticker', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'OI', 'Type', 'Strike', 'Expiry', 'Contract_Monthly', 'Contract_Weekly'])
options_dataframe['Datetime']=options_dataframe['Date'] +' '+ options_dataframe['Time']
print(options_dataframe.head())

['BANKNIFTY05JAN1718900CE.NFO', '2017-01-02', '09:15:59', '7.0', '12.0', '5.75', '5.8', '2360.0', '37160.0', 'CE', '18900.0', '2017-01-05', 'W', 'I']
['BANKNIFTY05JAN1719000CE.NFO', '2017-01-02', '09:15:59', '4.4', '6.6', '4.15', '4.2', '5680.0', '120760.0', 'CE', '19000.0', '2017-01-05', 'W', 'I']
['BANKNIFTY05JAN1719100CE.NFO', '2017-01-02', '09:15:59', '3.0', '3.0', '3.0', '3.0', '160.0', '9960.0', 'CE', '19100.0', '2017-01-05', 'W', 'I']
['BANKNIFTY05JAN1719500CE.NFO', '2017-01-02', '09:15:59', '1.5', '1.5', '1.5', '1.5', '80.0', '24160.0', 'CE', '19500.0', '2017-01-05', 'W', 'I']
['BANKNIFTY05JAN1718800CE.NFO', '2017-01-02', '09:15:59', '11.35', '13.9', '8.75', '9.2', '5480.0', '104640.0', 'CE', '18800.0', '2017-01-05', 'W', 'I']
                        Ticker        Date      Time   Open  High   Low Close  \
0  BANKNIFTY05JAN1718900CE.NFO  2017-01-02  09:15:59    7.0  12.0  5.75   5.8   
1  BANKNIFTY05JAN1719000CE.NFO  2017-01-02  09:15:59    4.4   6.6  4.15   4.2   
2  BANKNIFTY

In [11]:
#getting spot price and nearest strike price from following function (returns None if time not between 10:30AM and 3:20 PM)
def prices(line):
    try: 
        time_string = line[1]+' ' + line[2]  # concatenating strings of date and time to get format: YYYY-MM-DD HH:MM:SS
        # Convert string to datetime object
        time_obj = datetime.strptime(time_string, "%Y-%m-%d %H:%M:%S")

        # Convert datetime object to Unix time
        current_time = int(time_obj.timestamp())#converting time to unix timestamp
        start_time= int(datetime.strptime(line[1]+" 10:29:59", "%Y-%m-%d %H:%M:%S").timestamp())#to start at 10:30:00 latest data required would be of 10:29 AM
        end_time= int(datetime.strptime(line[1]+" 15:19:59", "%Y-%m-%d %H:%M:%S").timestamp())
        if current_time >=start_time and current_time<=end_time:
            spot_price=float(line[6])# taking closing price as the spot price
            strike_price= int(round(spot_price/100)*100) #finding nearest strike price to the spot
            return time_string, spot_price,strike_price
        else: 
            return None
    except ValueError:
        return

In [12]:
#put in the strike price and time(unix timestamp) and get all the available options 
#CONVERT INTO PANDAS DATAFRAME
def options(options_dataframe, current_time, strike_price):
    available_options = options_dataframe.loc[(options_dataframe['Datetime'] == current_time) & (options_dataframe['Strike'].astype(float).astype(int)==strike_price)]
    return available_options.values.tolist()

In [13]:
#simulating trade starting
def trade(row):
    if row[2][0:2]=='09':#when new day starts
        TRADE_ACTIVE=False
        return None
    if int(row[2].replace(':',''))==102959:
        TRADE_ACTIVE=True
        current_time, spot_price,strike_price=prices(row)
        atm_options=options(options_dataframe, current_time, strike_price)
        otm_options=options(options_dataframe, current_time, int(round((1.02*strike_price)/100)*100)) #finding nearest strike price 2% above atm strike price
        otm_options.extend(options(options_dataframe, current_time, int(round((0.98*strike_price)/100)*100))) #finding nearest strike price 2% below atm strike price)
        #print(atm_options)
        #print(otm_options)
        premium_collected=float(atm_options[0][6])+ float(atm_options[1][6])
        premium_paid=0
        
        #MAKE A START TRADE FUNCTION to return the initial net premium and which starts on every new day and ends on hitting stoploss or 3:20PM calculate all required performance stats in it
        for option in otm_options:
            if (option[0][-6:-4]=='CE' and float(option[0][-11:-6])>strike_price) or (option[0][-6:-4]=='PE' and float(option[0][-11:-6])<strike_price): #selecting CE of higher strike price and PE of lower strike price]
                premium_paid+=float(option[6])
        net_premium=premium_collected-premium_paid
        return net_premium

In [16]:
#sell and calculate net premiums
#keep updating premium
#calculate all required performance stats for the day and store it in lists or arrays
def report():
    trade_report=[]
    net_premium=0
    curr_net_premium=0
    for row in tqdm(futures_data):
        try:
            current_time, spot_price,strike_price=prices(row)
            atm_options=options(options_dataframe, current_time, strike_price)
            otm_options=options(options_dataframe, current_time, int(round((1.02*strike_price)/100)*100)) #finding nearest strike price 2% above atm strike price
            otm_options.extend(options(options_dataframe, current_time, int(round((0.98*strike_price)/100)*100))) #finding nearest strike price 2% below atm strike price)
            if atm_options==[]:
                if net_premium==0:
                    percent_return=0
                    continue
                else:
                    print('ending trade due to lack of atm options data')
                    percent_return=100*(net_premium-curr_net_premium)/net_premium
                    PROFIT=True if percent_return>=0 else False
                    trade_report.append([row[1],percent_return,PROFIT])
                    continue
            if trade(row)!=None:
                net_premium=trade(row)
            curr_premium_collected=float(atm_options[0][6])+ float(atm_options[1][6])
            curr_premium_paid=0
            hedge_num=0
            for option in otm_options:
                if (option[0][-6:-4]=='CE' and float(option[0][-11:-6])>strike_price) or (option[0][-6:-4]=='PE' and float(option[0][-11:-6])<strike_price): #selecting CE of higher strike price and PE of lower strike price]
                    hedge_num+=1
                    curr_premium_paid+=float(option[6])
            if hedge_num<2:
                print('ending trade due to lack of otm options data so cant take appropriate hadges')
                percent_return=100*(net_premium-curr_net_premium)/net_premium
                PROFIT=True if percent_return>=0 else False
                trade_report.append([row[1],percent_return,PROFIT])
                continue
            curr_net_premium=curr_premium_collected-curr_premium_paid
            if (curr_net_premium>=1.3*net_premium) or (curr_net_premium<=0.2*net_premium) or (row[2].strip()[0:5]=="15:19"):
                if net_premium==0:
                    percent_return=0
                else:
                    percent_return=100*(net_premium-curr_net_premium)/net_premium
                PROFIT=True if percent_return>=0 else False
                trade_report.append([row[1],percent_return,PROFIT])
        except TypeError:#basically when NoneType is returned by prices function
            continue
    return trade_report

In [18]:

#generating trade report
trade_report=report()
try: 
    average_profit= sum([sub_array[1] for sub_array in trade_report if sub_array[2]])/len([sub_array[1] for sub_array in trade_report if sub_array[2]])
except ZeroDivisionError:
    print("no profit making day in given data")
try:
    average_loss= sum([sub_array[1] for sub_array in trade_report if not sub_array[2]])/len([sub_array[1] for sub_array in trade_report if not sub_array[2]])
except ZeroDivisionError:
    print('no loss making day')
try:
    average_return=sum([sub_array[1] for sub_array in trade_report])/len(trade_report)
except ZeroDivisionError:
    print('data not enough to generate trade report for any day')
max_profit = max([sub_array[1] for sub_array in trade_report if sub_array[2]]) if [sub_array[1] for sub_array in trade_report if sub_array[2]] else None
max_loss= min([sub_array[1] for sub_array in trade_report if not sub_array[2]]) if [sub_array[1] for sub_array in trade_report if not sub_array[2]] else None
total_return=sum([sub_array[1] for sub_array in trade_report])
print(average_profit,average_loss,average_return,max_profit,max_loss,total_return)

  3%|▎         | 308/10000 [05:35<18:06,  8.92it/s]  

ending trade due to lack of otm options data so cant take appropriate hadges


  3%|▎         | 310/10000 [05:38<29:48,  5.42it/s]

ending trade due to lack of otm options data so cant take appropriate hadges


  3%|▎         | 314/10000 [05:44<1:20:51,  2.00it/s]

...

ending trade due to lack of otm options data so cant take appropriate hadges


 98%|█████████▊| 9777/10000 [3:13:18<06:12,  1.67s/it]

ending trade due to lack of otm options data so cant take appropriate hadges


100%|██████████| 10000/10000 [3:19:14<00:00,  1.20s/it]

20.629993770866044 -11.034066112626428 17.710365856252036 91.90110826939471 -35.84158415841585 16134.143295045604
